## Extracción y almacenamiento de datos

### Justificación de elección de API:

Para este trabajo decidí utilizar la API de JCDecaux Developers porque ofrece datos estáticos y dinámicos. Por un lado, proporciona información estática sobre cada contrato, lo cual es útil para realizar una extracción full que obtenga los metadatos de las estaciones de bicicletas. Por otro lado, ofrece datos en tiempo real sobre el estado de cada estación, incluyendo bicicletas disponibles, si se debe pagar, si está abierta, entre otros, los cuales se pueden obtener mediante extracción incremental. A partir de esto, utilcé dos endpoints en particular:
- **Extracción full:** Elegí el endpoint que ofrece los metadatos estáticos de las estaciones de cada contrato, ya que tiene gran cantidad de información que se puede limpiar y procesar luego.
- **Extracción incremental:** Seleccioné un endpoint que me permitió recolectar la información en tiempo real de cada estación, siendo útil para analizar tendencias futuras.

### Desarrollo del trabajo práctico:

##### Instalación de librerias:

In [37]:
!pip install requests
!pip install deltalake
!pip install pyarrow

##### Importación de librerias y módulos:

In [38]:
import requests
import pandas as pd
import json
import pyarrow as pa
import math
from datetime import datetime, timezone
from deltalake import write_deltalake, DeltaTable
from deltalake.exceptions import TableNotFoundError
from configparser import ConfigParser


##### Definición de funciones:

In [39]:
# Manejo de json

def get_last_update (file_path):

    try:
        #with es util porque permite simplificar el manejo de archivos, conexiones, db al asegurarse que se usan y liberan de forma correcta, incluso si hay errores.

        with open(file_path, "r") as file:
            ultimo_update = json.load(file) # leo el archivo json
            return ultimo_update["last_updated"]

    except FileNotFoundError:
        raise FileNotFoundError(f"El archivo JSON en la ruta {file_path} no existe.") # propago el error y lo resuelvo en otro lado!

    except json.JSONDecodeError:
        raise json.JSONDecodeError(f"El archivo JSON en la ruta {file_path} no es válido.")

    except KeyError as k:
        raise KeyError(str(k))


def update_last_update (file_path, last_updated):

    try:
        with open(file_path, "w") as file:
            json.dump({"last_updated": last_updated.isoformat()}, file, indent=4)

    except FileNotFoundError:
        raise FileNotFoundError(f"El archivo JSON en la ruta {file_path} no existe.") # propago el error y lo resuelvo en otro lado!

# Extraccion de datos

def get_data_stations(url, params=None):

    try:
        response = requests.get(url,params=params, timeout = 10)
        response.raise_for_status() # excepcion que captura el except
        data = response.json()
        return create_data_frame(data)

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener los datos. Código de error: {e}")
        return pd.DataFrame()

def create_data_frame(json_data):

    try:
        df = pd.json_normalize(json_data)
        return df

    except Exception as e:
        print(f"Se produjo un error en la construcción del DataFrame: {e}")
        return pd.DataFrame()

def incremental_extraction(url, file_path, params=None):

    try:

        last_update_json = get_last_update(file_path=file_path)
        last_update = pd.to_datetime(last_update_json, utc=True)

        df = get_data_stations(url, params=params)
        if df is None:
            print("No se pudo construir el DataFrame.")
            return pd.DataFrame()

        fechas_convertidas = []
        for f in df["last_update"]:
            if f is not None and not math.isnan(f):
                ft = datetime.fromtimestamp(f / 1000, tz=timezone.utc)
                fechas_convertidas.append(ft)
            else:
                fechas_convertidas.append(pd.NaT) #fecha nula en caso de ser null

        df["last_update"] = fechas_convertidas

        df_incremental = df[df["last_update"] > last_update]

        if df_incremental.empty:
            print("No hay nuevas actualizaciones desde la última consulta")
            return pd.DataFrame()

        max_timestamp = df["last_update"].max()
        update_last_update(file_path=file_path, last_updated=max_timestamp)
        return df_incremental

    except Exception as e:
        print(f"Se produjo un error en la extracción incremental {e}")
        return pd.DataFrame()


# Almacenamiento de datos

def save_data_as_delta(df, path, mode="overwrite", partition_cols=None):

    write_deltalake(path, df, mode = mode, partition_by = partition_cols)

def merge_new_data_as_delta(new_data, data_path, predicate, partition_cols=None):

    try:
        dt = DeltaTable(data_path)
        data_pa = pa.Table.from_pandas(new_data)
        dt.merge(
            source=data_pa,
            source_alias="source",
            target_alias="target",
            predicate=predicate
        ) \
        .when_matched_update_all() \
        .when_not_matched_insert_all() \
        .execute()
    except TableNotFoundError:
      save_data_as_delta(new_data, data_path, partition_cols=partition_cols)


#### Extracción Full:

Los metadatos de las estaciones se obtienen mediante una extracción full, ya que son datos estáticos que cambian muy poco o casi nunca. Por eso, no es necesario actualizarlos constantemente y una extracción full realizada con cierta frecuencia basta para mantenerlos actualizados.

In [40]:
# Obtengo api-keys

parser = ConfigParser()
parser.read("pipeline.conf")
api_key = parser["api-credentials"]["api-key"]

urlBase = "https://api.jcdecaux.com/vls/v1"

In [41]:
paramsEstatico = {
        "apiKey" : api_key
    }

endpointEstatico = "contracts"

urlEstatica = f"{urlBase}/{endpointEstatico}"

df_estatico = get_data_stations(urlEstatica, paramsEstatico)


#### Extracción Incremental:

En primer lugar, utilicé un método stateful, es decir, almacené el último update en un archivo JSON. Esto se debe a que los datos se actualizan en tiempo real, por lo que de esta forma siempre se obtiene solo la información nueva o modificada desde la última consulta. Para la extracción incremental, se consulta ese último update guardado y se solicitan únicamente los datos que superen ese estado.

In [42]:
endpointDinamico = "stations"

paramsDinamico = {
    "contract": "lyon",
    "apiKey" : api_key
}

urlDinamica= f"{urlBase}/{endpointDinamico}"

df_dinamico = incremental_extraction(urlDinamica, "metadata/metadata.json", paramsDinamico)

#### Almacenamiento de Datos - Extracción Full

En la extracción full, se sobreescriben todos los datos directamente para evitar la existencia de registros duplicados. Esto garantiza que siempre podamos ver el estado más actualizado.

In [43]:
bronze_dir = "datalake/bronze/luchtmeetnet_api"

In [44]:
contracts_dir = f"{bronze_dir}/{endpointEstatico}"

merge_new_data_as_delta(df_estatico, contracts_dir, "target.name = source.name")

In [45]:
dt = DeltaTable("datalake/bronze/luchtmeetnet_api/contracts")
df = dt.to_pandas()
df.sort_values(by= "name")


,name,commercial_name,cities,country_code
12,amiens,Velam,[Amiens],FR
26,besancon,None,None,None
25,brisbane,CityCycle,[Brisbane],AU
10,bruxelles,villo,"[Anderlecht, Berchem-Sainte-Agathe, Bruxelles,...",BE
20,cergy-pontoise,Velo2,"[Cergy, Courdimanche, Eragny-sur-Oise, Neuvill...",FR
19,creteil,Cristolib,[Créteil],FR
4,dublin,dublinbikes,[Dublin],IE
1,jcdecauxbike,None,None,None
14,lillestrom,Bysykkel,[Lillestrom],NO
15,ljubljana,Bicikelj,[Ljubljana],SI


#### Almacenamiento de Datos - Extracción Incremental

Decidí registrar todos los datos que obtengo de la API en tiempo real, guardando un historial completo del estado de cada estación en cada fecha y hora específica. Es decir, las actualizaciones se realizan considerando cada combinación de fecha y hora, evitando duplicados por fecha y hora. Si bien esto puede generar algunos datos repetidos, creo que es una estrategia adecuada porque me permite conocer el estado de cada estación en cada momento. Esto facilita analizar cambios a lo largo del tiempo y detectar tendencias.

In [46]:
stations_dir = f"{bronze_dir}/{endpointDinamico}"

if not df_dinamico.empty:

    df_dinamico["last_update"] = pd.to_datetime(df_dinamico.last_update)

    df_dinamico["fecha"] = df_dinamico.last_update.dt.date

    df_dinamico["hora"] = df_dinamico.last_update.dt.hour

    merge_new_data_as_delta(df_dinamico, stations_dir, predicate= "target.number = source.number AND target.fecha = source.fecha AND target.hora=source.hora", partition_cols=["fecha", "hora"])

In [47]:
dt = DeltaTable("datalake/bronze/luchtmeetnet_api/stations")
df = dt.to_pandas()
df.sort_values(by= "number")


,number,contract_name,name,address,banking,bonus,bike_stands,available_bike_stands,available_bikes,status,last_update,position.lat,position.lng,fecha,hora,__index_level_0__
669,555,lyon,0-555 - ATELIER VÉLO'V,,False,False,3,3,0,OPEN,2025-08-10 15:42:19+00:00,45.780403,4.904869,2025-08-10,15,201
133,555,lyon,0-555 - ATELIER VÉLO'V,,False,False,3,3,0,OPEN,2025-08-10 16:02:29+00:00,45.780403,4.904869,2025-08-10,16,201
163,1001,lyon,1001 - TERREAUX / TERME,Angle rue d'Algérie,False,False,16,13,1,OPEN,2025-08-10 16:02:47+00:00,45.767736,4.832114,2025-08-10,16,238
553,1001,lyon,1001 - TERREAUX / TERME,Angle rue d'Algérie,False,False,16,13,1,OPEN,2025-08-10 15:49:38+00:00,45.767736,4.832114,2025-08-10,15,238
232,1002,lyon,1002 - OPÉRA,Angle rue Serlin - Angle place de la comédie,False,False,41,40,0,OPEN,2025-08-10 16:03:18+00:00,45.767611,4.836619,2025-08-10,16,340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,32001,lyon,32001 - COUZON - CENTRE,,False,False,17,12,5,OPEN,2025-08-10 15:46:20+00:00,45.846034,4.832409,2025-08-10,15,2
396,33001,lyon,33001 - ALBIGNY - GARE,,False,False,18,3,15,OPEN,2025-08-10 15:58:34+00:00,45.874978,4.833024,2025-08-10,15,364
379,34001,lyon,34001 - NEUVILLE - QUAI PASTEUR,,False,False,20,4,16,OPEN,2025-08-10 15:56:54+00:00,45.876622,4.838578,2025-08-10,15,307
616,34002,lyon,34002 - NEUVILLE - STADE,,False,False,15,7,8,OPEN,2025-08-10 15:46:57+00:00,45.870877,4.839595,2025-08-10,15,357
